# Overview and Tensors
---
In thie notebook, I'll describe 
- Overview about the tensorflow
- Execute operations using tf.Session()
- Tensors
    - constant
    - Variable
    - placeholder

## Overview about the tensorflow
- Tensorflow calculates various kinds of data operations using a computational graph.
- The defined computational graph is executed in tf.Session().
    - Basically tensorflow treats computations in define-and-run style.
    - The return values of the tf.Session.run() are numpy array.
    - Recently, tensorflow supports define-by-run style, that is "Eager Execution".

In [1]:
# packages
import tensorflow as tf
import numpy as np

print("tensorflow version: ", tf.__version__)
print("numpy version: ", np.__version__)

tensorflow version:  1.15.2
numpy version:  1.18.2


## Execute operations using tf.Session()
Pipeline:
- Build a computational graph.
- Run session.

In [2]:
# clear graph
tf.reset_default_graph()

# build graph
test_const = tf.constant(13.5, tf.float16)

print(test_const) # In this line, test_cost is a abstract constant tensor.

# run session
with tf.Session() as sess:
    out = sess.run(test_const) # In this line, test_const makes concrete data.
    print(out)

Tensor("Const:0", shape=(), dtype=float16)
13.5


## Tensors

### [tf.constant](https://www.tensorflow.org/api_docs/python/tf/constant)
- A constant tensor is a fixed tensor in a computation graph operation.
- When we try to change values of constant tensors, tensorflow raises exception.

```python
tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)
```

Args:
- value: A constant value (or list) of output type dtype.
- dtype: The type of the elements of the resulting tensor.
- shape: Optional dimensions of resulting tensor.
- name: Optional name for the tensor.
- verify_shape: Boolean that enables verification of a shape of values.
  
Returns:  
- A Constant Tensor.  
  
Raises:  
- TypeError: if shape is incorrectly specified or unsupported.  

__Examples:__  

In [3]:
# clear graph
tf.reset_default_graph()

# 0-dimensional scalor tensor (int32)
const1 = tf.constant(1)
print("const1: ", const1)

# 1-dimensional vector tensor
const2 = tf.constant([1,2,3])
print("const2: ", const2)

# 2-dimensional matrix tensor
const3 = tf.constant([[1,2,3],[4,5,6]])
print("const3: ", const3)

# And so on.

with tf.Session() as sess:
    print(sess.run(const1))
    print(sess.run(const2))
    print(sess.run(const3))

const1:  Tensor("Const:0", shape=(), dtype=int32)
const2:  Tensor("Const_1:0", shape=(3,), dtype=int32)
const3:  Tensor("Const_2:0", shape=(2, 3), dtype=int32)
1
[1 2 3]
[[1 2 3]
 [4 5 6]]


## [tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)

- A variable is a changeable tensor.
- We can define a variable like following:
    - `val = tf.Variable(<initial-value>, name=<optional-name>)`
    - `my_variable = tf.get_variable("my_variable", shape=[1, 2, 2])`
- In general, variable tensors are treated as optimization targets.
- Variables must be initialized in the session.
    - If we want to initialize all variables, use `sess.run(tf.global_variables_initiaizer()) `.
    - If we want to initialize variables individually, use `sess.run(val_name.initializer)`.
    
```python
tf.Variable(initial_value=None, trainable=True, collections=None, validate_shape=True, caching_device=None, name=None, variable_def=None, dtype=None, expected_shape=None, import_scope=None, constraint=None, use_resource=None, synchronization=tf.VariableSynchronization.AUTO, aggregation=tf.VariableAggregation.NONE)
```

Args:
- initial_value: A Tensor, or Python object convertible to a Tensor, which is the initial value for the Variable. The initial value must have a shape specified unless validate_shape is set to False. Can also be a callable with no argument that returns the initial value when called. In that case, dtype must be specified. (Note that initializer functions from init_ops.py must first be bound to a shape before being used here.)
- trainable: If True, the default, also adds the variable to the graph collection GraphKeys.TRAINABLE_VARIABLES. This collection is used as the default list of variables to use by the Optimizer classes.
- collections: List of graph collections keys. The new variable is added to these collections. Defaults to [GraphKeys.GLOBAL_VARIABLES].
- validate_shape: If False, allows the variable to be initialized with a value of unknown shape. If True, the default, the shape of initial_value must be known.
- caching_device: Optional device string describing where the Variable should be cached for reading. Defaults to the Variable's device. If not None, caches on another device. Typical use is to cache on the device where the Ops using the Variable reside, to deduplicate copying through Switch and other conditional statements.
- name: Optional name for the variable. Defaults to 'Variable' and gets uniquified automatically.
- variable_def: VariableDef protocol buffer. If not None, recreates the Variable object with its contents, referencing the variable's nodes in the graph, which must already exist. The graph is not changed. variable_def and the other arguments are mutually exclusive.
- dtype: If set, initial_value will be converted to the given type. If None, either the datatype will be kept (if initial_value is a Tensor), or convert_to_tensor will decide.
- expected_shape: A TensorShape. If set, initial_value is expected to have this shape.
- import_scope: Optional string. Name scope to add to the Variable. Only used when initializing from protocol buffer.
- constraint: An optional projection function to be applied to the variable after being updated by an Optimizer (e.g. used to implement norm constraints or value constraints for layer weights). The function must take as input the unprojected Tensor representing the value of the variable and return the Tensor for the projected value (which must have the same shape). Constraints are not safe to use when doing asynchronous distributed training.
- use_resource: whether to use resource variables.
- synchronization: unused
- aggregation: unused

Raises:
- ValueError: If both variable_def and initial_value are specified.
- ValueError: If the initial value is not specified, or does not have a shape and validate_shape is True.
- RuntimeError: If eager execution is enabled.

__Examples:__

In [4]:
# clear graph
tf.reset_default_graph()

# define variables
val1 = tf.Variable(tf.constant([1,2]))
val2 = tf.Variable(tf.constant([3,4]), name="val2")
val3 = tf.get_variable("val3", [2, 2], initializer=tf.initializers.he_normal())
print(val1)
print(val2)
print(val3)

print("------------Session1------------")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # initalize all variable
    print(sess.run(val1))
    print(sess.run(val2))
    print(sess.run(val3))

print("------------Session2------------")
with tf.Session() as sess:
    sess.run(val1.initializer) # initalize individually
    sess.run(val2.initializer)
    sess.run(val3.initializer)
    print(sess.run(val1))
    print(sess.run(val2))
    print(sess.run(val3))

<tf.Variable 'Variable:0' shape=(2,) dtype=int32_ref>
<tf.Variable 'val2:0' shape=(2,) dtype=int32_ref>
<tf.Variable 'val3:0' shape=(2, 2) dtype=float32_ref>
------------Session1------------
[1 2]
[3 4]
[[-0.08191002 -0.1338891 ]
 [ 0.5173946   0.28501067]]
------------Session2------------
[1 2]
[3 4]
[[-0.19725624 -0.06469614]
 [-2.156629   -0.41741645]]


## [tf.placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder)
- A placeholder is something like empty box for a tensor that will be fed.
- We define the dtype and shape of the placeholder when we build a graph, then we fed tensor using `feed_dit` in a session.

``` python
tf.placeholder(dtype, shape=None, name=None)
```
Args:
- dtype: The type of elements in the tensor to be fed.
- shape: The shape of the tensor to be fed (optional). If the shape is not specified, you can feed a tensor of any shape.
- name: A name for the operation (optional).

Returns:
- A Tensor that may be used as a handle for feeding a value, but not evaluated directly.

Raises:
- RuntimeError: if eager execution is enabled  
__Note that we can't use placeholder in egaer execution__  

__Examples:__

In [5]:
# clear graph
tf.reset_default_graph()

place1 = tf.placeholder(tf.int32, shape=(2,2))
input_data = np.array([[1,2],[3,4]])

print("place1: ", place1)
print("input_data: ", input_data)

with tf.Session() as sess:
    out = sess.run(place1, feed_dict={place1 : input_data})
    print(out)

place1:  Tensor("Placeholder:0", shape=(2, 2), dtype=int32)
input_data:  [[1 2]
 [3 4]]
[[1 2]
 [3 4]]
